<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2018


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>

<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> Normalization </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b> October 1, 2018</b></p>

<hr style='height:5px;border:none' />

# Brief introduction to Nipype
<hr style="height:1px;border:none" />

## Steps in fMRI data analysis

Although there are different software tools designed for fMRI data analysis, the steps required for such an analysis is very similar for the most part. 

<img style='width: 650px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Nipype_fMRIOverview.png?raw=true' alt='fMRI processing steps'/>

Different software packages implements these steps to analyze fMRI data. So, in theory, it is possible to mix different steps from different software tools but you can still implement all the necessary steps.

## Why nipype?

Different fMRI analysis software tools have their own scripting capability. In other words, each package lets you write a script (a series of instructions) to implement the entire analysis pipeline from the beginning to the end. This is useful especially you have to run the same pre-processing and analysis steps for multiple subjects. Analyzing multiple subjects' data using a GUI will increase the risk of human error (clicking a wrong button, selecting a wrong file, entering wrong parameters, etc.). 

The syntax to script a processing pipeline is different in different packages. For a starter, in SPM you write a script in MATLAB, whereas in FSL, you need to code in a shell script. **Nipype** enables you to script different steps in different software tools in Python, so that mixing of different steps very easy.

<img style='width: 650px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Nipype_WorkflowEg.png?raw=true' alt='Nipype workflow example'/>

Here is an example of how `nipype` implements different steps. The diagram such as this one represents a **workflow**, a series of tasks organized into a diagram, with output from one task fed into the following task. Here, each task is represented as a **node**. A node may take a certain input(s) and produces an output. 

As the first example in **`nipype`**, we will implement normalization of structural MRI data. 

# Normalization - Theory
<hr style="height:1px;border:none" />

* What is normalization?
* Linear
   * 12 parameter transformations
* Non-linear
   * Various warping methods
* Cost function
* Regularization

# Preparing for normalization
<hr style="height:1px;border:none" />

* Skull striping
* Segmentation

# Normalization with Nipype
<hr style="height:1px;border:none" />

* With FSL
* With SPM
* With ANTs

# Normalization with a template
<hr style="height:1px;border:none" />

* Pediatric template
* Study-specific template

# Normalization of 
<hr style="height:1px;border:none" />

* Pediatric template
* Study-specific template